In [2]:
# Data & Preprocessing
import numpy as np
import pandas as pd
import tensorflow as tf
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Add

In [4]:
# fetch dataset 
covertype = fetch_ucirepo(id=31) 
  
# data (as pandas dataframes) 
X = covertype.data.features 
y = covertype.data.targets 
  
# metadata 
print(covertype.metadata) 
  
# variable information 
print(covertype.variables) 

{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [6]:
# Preprocessing
# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-validation split
y_binary = (y.iloc[:, 0] == 1).astype(int)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42)

In [8]:
# Build model
inputs = Input(shape=(X.shape[1],), name="input_layer")

# Initial dense layer
x_1 = Dense(32, activation='relu', name="initial_dense")(inputs)

# Residual block
res = Dense(32, activation='relu', name="res_dense_1")(x_1)
res = Dense(32, activation='relu', name="res_dense_2")(res)
res_out = Add(name="Residual_Add")([x_1, res])

# Intermediate layer
x_2 = Dense(16, activation='relu', name="intermediate_dense")(res_out)

# Skip connection from x_1 to output
skip = Dense(16, activation='relu', name="Skip_Proj")(x_1)
skip_out = Add(name="Skip_Add")([x_2, skip])

# Final layer
output = Dense(1, activation='sigmoid', name="Output")(skip_out)

# Compile model
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 54)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ initial_dense       │ (None, 32)        │      1,760 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_dense_1 (Dense) │ (None, 32)        │      1,056 │ initial_dense[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_dense_2 (Dense) │ (None, 32)        │      1,056 │ res_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual_Add (Add)  │ (None, 32)        │          0 │ initial_dense[0]… │
│                     │                   │            │ res_dense_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intermediate_dense  │ (None, 16)        │        528 │ Residual_Add[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Skip_Proj (Dense)   │ (None, 16)        │        528 │ initial_dense[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Skip_Add (Add)      │ (None, 16)        │          0 │ intermediate_den… │
│                     │                   │            │ Skip_Proj[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output (Dense)      │ (None, 1)         │         17 │ Skip_Add[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,945 (19.32 KB)

 Trainable params: 4,945 (19.32 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Save model for Netron
model.save("model_with_residuals.h5")

In [11]:
# Select a batch of 128 samples
X_batch = X_train[:128]
y_batch = y_train[:128]

# Train only on this batch
history = model.fit(X_batch, y_batch, epochs=500, verbose=0)

# Final training loss
train_loss = model.evaluate(X_batch, y_batch, verbose=0)[0]

# Validation loss
val_loss = model.evaluate(X_val, y_val, verbose=0)[0]

# printing results
print(f"\nNumber of parameters: {model.count_params()}")
print(f"Final training loss: {train_loss:.4f}")
print(f"Final validation loss: {val_loss:.4f}")


Number of parameters: 4945
Final training loss: 0.0000
Final validation loss: 4.3092
